In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import numpy as np

In [5]:
# 사전에 수집한 네이버 제품목록 시트를 불러온다.
df = pd.read_excel('./naver_0901.xlsx')

In [6]:
links = df.link
s_links = list(set(links))

In [7]:
ch = []
df1 = pd.DataFrame({'texts':ch, 'grade':ch, 'day':ch, 'link':ch})

In [8]:
#리뷰 수집 함수. 리뷰 텍스트, 날짜, 별점을 수집.
def crawling(soup):
    review = soup.find('div', {'class': '_2y6yIawL6t'})
    reviews = review.find_all('li', {'class': '_2389dRohZq'})
    star = review.find_all('em', {'class': '_15NU42F3kT'})
    
    
    datas = []

    for i in range(len(star)):
        re = reviews[i].find_all('span', {'class':'_3QDEeS6NLn'})
        datas.append(re[len(re)-1].get_text())    
        datas.append(star[i].get_text())
        datas.append(re[0].get_text())
        
    return(datas)

In [9]:

#리뷰 페이지 이동 함수
def da1(all_data):
    
    count = 0 
    while True:
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        bar = soup.find('div', {'class': '_3YOQJDR2-0'})
        now = int(bar.find('a', {'aria-current': 'true'}).get_text())


        all_data = all_data + crawling(soup)
        
        
        if len(all_data) >= 900:
            break
        
        
        goto = (now%10)+2
        if goto == 2:
            goto = 12
            
        try: 
            buttons = '//*[@id="REVIEW"]/div/div[3]/div/div[2]/div/div/a[{}]'.format(goto)
            next_b = driver.find_element_by_xpath(buttons)
            next_b.click()
            time.sleep(2)            
        except:
            break
    print(len(all_data))        
    return(all_data)

In [10]:
#사전에 조사된 페이지 주소에서 리뷰 데이터 수집 시작.
for r_link in s_links:
    all_data = []
    driver = webdriver.Chrome(executable_path='chromedriver')
    driver.get(r_link)
    check1 = driver.current_url
    time.sleep(2)
    
    review = 0
    
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    
    
    try:
        #해당 제품의 리뷰 갯수 조회.
        extra = soup.find_all('strong', {'class': '_2pgHN-ntx6'})
        review = extra[0].get_text()
        review = int(re.sub(r'[^0-9]', '', review))
    except:
        review = 0
    print(r_link)
    print(review)   
    #리뷰 갯수가 30개 이상일 경우 리뷰 수집.
    if review > 30:
        all_data = da1(all_data)
        
    driver.close()
    dl = len(all_data)//3


    count = 0 
    for i in range(0, dl):    
        d_li = []
        num = i*3
        d_li.append(all_data[num])
        d_li.append(all_data[num+1])
        d_li.append(all_data[num+2])
        d_li.append(check1)
        df1.loc[len(df1)] = d_li
    
    time.sleep(2)

https://smartstore.naver.com/jmhouse0507/products/5360894388
243
720
https://smartstore.naver.com/t0316563449/products/4831756132
3798
900
https://smartstore.naver.com/jhstoremall/products/4197080006?n_media=11068&n_rank=95&n_ad_group=grp-a001-02-000000016483351&n_ad=nad-a001-02-000000100802471&n_campaign_type=2&n_mall_id=ncp_1nloe9_01&n_mall_pid=4197080006&n_ad_group_type=2
978
900
https://smartstore.naver.com/infobell/products/3876483413
804
900
https://smartstore.naver.com/ssarijai/products/716167585?n_media=11068&n_rank=85&n_ad_group=grp-a001-02-000000008479133&n_ad=nad-a001-02-000000093055429&n_campaign_type=2&n_mall_id=ssarijai&n_mall_pid=716167585&n_ad_group_type=2
2017
900
https://smartstore.naver.com/heraparty/products/5076537205
2906
900
https://smartstore.naver.com/containerpeople/products/5276529184
84
312
https://smartstore.naver.com/light_market/products/5782450910
14
https://smartstore.naver.com/5000tteokbokki/products/5090576623?n_media=11068&n_rank=93&n_ad_group=grp-a0

In [67]:
# 페이지 주소를 기준으로 데이터프레임 조인
t_df= df[['name', 'price', 'review', 'img_url', 'product_num', 'brand', 'regi', 'link', 'market']]
t_df = t_df.drop_duplicates(['link'])
df_LEFT_JOIN = pd.merge(df2, t_df, left_on='link', right_on='link', how='left')



In [1]:
#날짜 추가

d_y = []
d_m = []
d_d = []
for i in df_LEFT_JOIN.day:
    i = i.replace(' ','')
    dd = i.split('-')
    d_y.append(dd[0])
    d_m.append(dd[1])
    d_d.append(dd[2])
    
df_LEFT_JOIN['year'] = d_y
df_LEFT_JOIN['month'] = d_m
df_LEFT_JOIN['date'] = d_d

df_LEFT_JOIN = df_LEFT_JOIN.drop(["day"], axis=1)

NameError: name 'df_LEFT_JOIN' is not defined

In [73]:
df_LEFT_JOIN.to_excel('naver_reviews.xls')

<ipython-input-73-ea1b643c6315>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_LEFT_JOIN.loc[:65532].to_excel('n_reviews_1.xls')
